In [1]:
import pandas as pd
import re
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense

train = pd.read_csv("../../../datasources/titanic/train.csv")
test = pd.read_csv("../../../datasources/titanic/test.csv")

Using TensorFlow backend.


In [2]:
familynames = []
for noms in test["Name"]:
    familynames.append(re.search('([A-Za-z0-9]*),\ ([A-Za-z0-9 ]*)\. (.*)', noms).group(1))
pdfamilynames = pd.DataFrame(familynames, columns = ['familynames'])
famsurv = test.join(pdfamilynames)

def dataprep(data):
    sexe = pd.get_dummies(data['Sex'], prefix='sex')
    cabin = pd.get_dummies(data['Cabin'].fillna('X').str[0], prefix='Cabin')
    # Age 
    age = data['Age'].fillna(data['Age'].mean())
    enfantf = []
    for i in range(data.shape[0]) :
        if age[i] < 9:
            enfantf.append(0)
        else:
            enfantf.append(1)
    enfantd = pd.DataFrame(enfantf, columns = ['Enfant'])
    
    # Titre (Master principalement)
    ismaster = []
    for noms in data["Name"]:
        if (re.search('([A-Za-z0-9]*),\ ([A-Za-z0-9 ]*)\. (.*)', noms).group(2) == "Master") :
            ismaster.append(1)
        else:
            ismaster.append(0)
    pdismaster = pd.DataFrame(ismaster, columns = ['isMaster'])

    emb = pd.get_dummies(data['Embarked'], prefix='emb')
    # Prix du billet / Attention une donnée de test n'a pas de Prix !
    faresc = pd.DataFrame(MinMaxScaler().fit_transform(data[['Fare']].fillna(0)), columns = ['Prix'])
    # Classe
    pc = pd.DataFrame(MinMaxScaler().fit_transform(data[['Pclass']]), columns = ['Classe'])

    dp = data[['SibSp', 'Parch']].join(pc).join(sexe).join(emb).join(faresc).join(cabin).join(age).join(pdismaster).join(enfantd)
    return dp

In [29]:
Xtrain = dataprep(train)
ytrain = train.Survived
# remove the Cabin with T value as this does not exist in the test dataset !
del Xtrain['Cabin_T']
Xtest = dataprep(test)

/opt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [30]:
Xtrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 20 columns):
SibSp         891 non-null int64
Parch         891 non-null int64
Classe        891 non-null float64
sex_female    891 non-null uint8
sex_male      891 non-null uint8
emb_C         891 non-null uint8
emb_Q         891 non-null uint8
emb_S         891 non-null uint8
Prix          891 non-null float64
Cabin_A       891 non-null uint8
Cabin_B       891 non-null uint8
Cabin_C       891 non-null uint8
Cabin_D       891 non-null uint8
Cabin_E       891 non-null uint8
Cabin_F       891 non-null uint8
Cabin_G       891 non-null uint8
Cabin_X       891 non-null uint8
Age           891 non-null float64
isMaster      891 non-null int64
Enfant        891 non-null int64
dtypes: float64(3), int64(4), uint8(13)
memory usage: 60.1 KB


In [31]:
model = Sequential()
model.add(Dense(units=Xtrain.shape[1], activation='relu', input_dim=Xtrain.shape[1]))
model.add(Dense(activation="relu", units=100, kernel_initializer="uniform"))
model.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_5 (Dense)              (None, 100)               2100      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 2,621
Trainable params: 2,621
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(Xtrain, ytrain, epochs=50, batch_size=30)

Epoch 1/50
891/891 [==============================] - 0s 238us/step - loss: 0.6648 - acc: 0.6072
Epoch 2/50
891/891 [==============================] - 0s 34us/step - loss: 0.6581 - acc: 0.6240
Epoch 3/50
891/891 [==============================] - 0s 31us/step - loss: 0.6518 - acc: 0.6308
Epoch 4/50
891/891 [==============================] - 0s 30us/step - loss: 0.6355 - acc: 0.6465
Epoch 5/50
891/891 [==============================] - 0s 30us/step - loss: 0.6071 - acc: 0.6902
Epoch 6/50
891/891 [==============================] - 0s 32us/step - loss: 0.5595 - acc: 0.7160
Epoch 7/50
891/891 [==============================] - 0s 30us/step - loss: 0.5185 - acc: 0.7710
Epoch 8/50
891/891 [==============================] - 0s 32us/step - loss: 0.4821 - acc: 0.7778
Epoch 9/50
891/891 [==============================] - 0s 35us/step - loss: 0.4568 - acc: 0.7901
Epoch 10/50
891/891 [==============================] - 0s 35us/step - loss: 0.4565 - acc: 0.7924
Epoch 11/50
891/891 [=================

In [40]:
scores = model.evaluate(Xtrain, ytrain)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

891/891 [==============================] - 0s 12us/step

acc: 83.95%


In [46]:
predictions = model.predict(Xtest)
predictions

array([[0.09803689],
       [0.6330723 ],
       [0.10034603],
       [0.088754  ],
       [0.45312175],
       [0.13981578],
       [0.7433976 ],
       [0.11237267],
       [0.80292535],
       [0.05954331],
       [0.08428693],
       [0.16276526],
       [0.9761257 ],
       [0.09077376],
       [0.9866574 ],
       [0.9057617 ],
       [0.15029013],
       [0.13289568],
       [0.6674241 ],
       [0.7216089 ],
       [0.22446108],
       [0.9370959 ],
       [0.9094516 ],
       [0.3010469 ],
       [0.9836848 ],
       [0.06980684],
       [0.98359454],
       [0.12999612],
       [0.34508485],
       [0.14519188],
       [0.11065146],
       [0.12385252],
       [0.4407675 ],
       [0.41165894],
       [0.44540563],
       [0.13699013],
       [0.66731215],
       [0.7091823 ],
       [0.09163293],
       [0.08992696],
       [0.09406134],
       [0.540379  ],
       [0.0712806 ],
       [0.81609154],
       [0.98424655],
       [0.09144172],
       [0.44458157],
       [0.104

In [52]:
print([1 if x >= 0.5 else 0 for x in predictions])

[0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 

In [24]:
final = pd.DataFrame()
final['PassengerId'] = test['PassengerId']
final['Survived'] = [1 if x >= 0.5 else 0 for x in predictions]
final.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [26]:
final.to_csv("result.csv", columns=["PassengerId", "Survived"], index=False)

Kaggle score : 0.75598